In [2]:
import sqlite3
import pandas as pd
import datetime
import load_database_functions
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import sqlite3
import re
con = sqlite3.connect('all_cbb_data_2-24_v1.db')
cur = con.cursor()


# DB Structure

## Tables and Column    
- Teams
    - espn_team_id
    - team_name
    - logo_url
    - team_page_url 
- Team_Week_Rankings
    - espn_team_id
    - week_number
    - ranking
- Team_Schedule
    - espn_team_id
    - opponent_espn_team_id
    - game_date
    - is_home
    - is_neutral_court
    - is_win
    - final_score
    - opponent_final_score
    - game_time
    - tv
    - cancelled_or_postponed
- Weeks
    - week_number
    - start_date_inclusive
    - end_date_exclusive
    - is_current_week

In [2]:
con = sqlite3.connect('all_cbb_data.db')
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE teams
               (espn_team_id integer, team_name text, logo_url text, team_page_url text)''')


cur.execute('''CREATE TABLE team_week_rankings
               (espn_team_id integer, week_number integer, ranking integer)''')

cur.execute('''CREATE TABLE team_schedule
               (espn_team_id integer, opponent_espn_team_id integer, game_date text, is_home integer, 
               is_neutral_court integer, is_win integer, final_score integer, opponent_final_score integer,
               game_time text, tv text, cancelled_or_postponed integer)''')

cur.execute('''CREATE TABLE weeks
               ( week_number integer, start_date_inclusive text, end_date_exclusive text, is_current_week integer)''')

season_start_date = datetime.date(2021,11,8)
season_end_date = datetime.date(2022,3, 14)

load_database_functions.create_weeks_table(con, season_start_date, season_end_date)
# Insert a row of data
#cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()

In [8]:

season_start_date = datetime.date(2021,11,8)
(season_start_date+ datetime.timedelta(days=14)).strftime("%a, %m-%d-%y")

season_start_date = datetime.date(2021,11,8)
season_end_date = datetime.date(2022,3, 14)

(season_end_date - season_start_date).days/7


18.0

In [7]:
df = pd.read_sql_query("SELECT * from teams", con)
df

,espn_team_id,team_name,logo_url,team_page_url


In [2]:
con = sqlite3.connect('all_cbb_data.db')
cur = con.cursor()
#cur.execute("INSERT INTO team_week_rankings VALUES (1,15,25)")

In [26]:
max_week = cur.execute('''SELECT max(week_number)
                FROM team_week_rankings''')

In [27]:
max_week.fetchone()[0]

15

In [3]:
con.close()

In [4]:

con = sqlite3.connect('all_cbb_data.db')
cur = con.cursor()

#teams_df, week_rank_df = load_database_functions.load_top25_teams(con,cur,2)
existing_teams_df, teams_df = load_database_functions.load_top25_teams(con, 2)



In [10]:
existing_teams_df

,espn_team_id,team_name,logo_url,team_page_url
0,2250,Gonzaga,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2250/gonzag...
1,26,UCLA,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/26/ucla-bruins
2,2305,Kansas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2305/kansas...
3,222,Villanova,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/222/villano...
4,251,Texas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/251/texas-l...
5,130,Michigan,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/130/michiga...
6,2509,Purdue,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2509/purdue...
7,239,Baylor,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/239/baylor-...
8,150,Duke,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/150/duke-bl...
9,96,Kentucky,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/96/kentucky...


In [11]:
teams_df

,espn_team_id,team_name,logo_url,team_page_url
0,2250,Gonzaga,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2250/gonzag...
1,26,UCLA,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/26/ucla-bruins
2,2305,Kansas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2305/kansas...
3,130,Michigan,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/130/michiga...
4,222,Villanova,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/222/villano...
5,2509,Purdue,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2509/purdue...
6,150,Duke,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/150/duke-bl...
7,251,Texas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/251/texas-l...
8,239,Baylor,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/239/baylor-...
9,356,Illinois,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/356/illinoi...


In [6]:
#teams
#team_week_rankings
df = pd.read_sql_query("SELECT * from teams", con)
df

,espn_team_id,team_name,logo_url,team_page_url
0,2250,Gonzaga,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2250/gonzag...
1,26,UCLA,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/26/ucla-bruins
2,2305,Kansas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2305/kansas...
3,222,Villanova,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/222/villano...
4,251,Texas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/251/texas-l...
5,130,Michigan,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/130/michiga...
6,2509,Purdue,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2509/purdue...
7,239,Baylor,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/239/baylor-...
8,150,Duke,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/150/duke-bl...
9,96,Kentucky,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/96/kentucky...


In [18]:
teams_df['espn_team_id'].sort_values()

19     120
3      130
6      150
17     153
21     179
18     194
20       2
4      222
0     2250
2     2305
10     235
8      239
14     248
11    2483
5     2509
7      251
1       26
16    2633
24      30
13     333
9      356
22      41
23      57
15       8
12      96
Name: espn_team_id, dtype: object

In [19]:
existing_teams_df['espn_team_id'].sort_values()

21       2
15       8
1       26
23      41
19      52
9       96
20     120
5      130
8      150
18     153
22     179
16     194
3      222
11     235
7      239
14     248
4      251
24     258
13     333
10     356
0     2250
2     2305
12    2483
6     2509
17    2633
Name: espn_team_id, dtype: int64

In [14]:
df['espn_team_id'].sort_values()

46       2
21       2
70       2
40       8
65       8
      ... 
6     2509
55    2509
42    2633
66    2633
17    2633
Name: espn_team_id, Length: 75, dtype: int64

In [7]:
teams_df

,espn_team_id,team_name,logo_url,team_page_url
0,2250,Gonzaga,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2250/gonzag...
1,26,UCLA,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/26/ucla-bruins
2,2305,Kansas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2305/kansas...
3,130,Michigan,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/130/michiga...
4,222,Villanova,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/222/villano...
5,2509,Purdue,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/2509/purdue...
6,150,Duke,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/150/duke-bl...
7,251,Texas,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/251/texas-l...
8,239,Baylor,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/239/baylor-...
9,356,Illinois,https://a.espncdn.com/combiner/i?img=/i/teamlo...,/mens-college-basketball/team/_/id/356/illinoi...


In [8]:
week_rank_df

,espn_team_id,week_number,ranking
0,2250,2,1
1,26,2,2
2,2305,2,3
3,130,2,4
4,222,2,5
5,2509,2,6
6,150,2,7
7,251,2,8
8,239,2,9
9,356,2,10


In [5]:
this_team_schedule_df = pd.read_html(load_database_functions.TEAM_SCHEDULE_URL.replace('TEAM_ID', str(2250)))[0]
this_team_schedule_df

,0,1,2,3,4,5,6,7
0,Regular Season,Regular Season,Regular Season,Regular Season,Regular Season,Regular Season,Regular Season,Regular Season
1,DATE,OPPONENT,RESULT,W-L (CONF),Hi Points,Hi Rebounds,Hi Assists,NaN
2,"Tue, Nov 9",vs Dixie State,W97-63,1-0 (0-0),Strawther 17,Holmgren 13,Watson 7,NaN
3,"Sat, Nov 13",vs 5 Texas,W86-74,2-0 (0-0),Timme 37,Timme 7,Nembhard 8,NaN
4,"Mon, Nov 15",vs Alcorn State,W84-57,3-0 (0-0),Strawther 18,Holmgren 6,Timme 6,NaN
5,"Fri, Nov 19",vs Bellarmine,W92-50,4-0 (0-0),Timme 25,Strawther 6,Nembhard 7,NaN
6,"Mon, Nov 22",vs Central Michigan *,W107-54,5-0 (0-0),Holmgren 19,Sallis 7,Bolton 5,NaN
7,"Tue, Nov 23",vs 2 UCLA *,W83-63,6-0 (0-0),Nembhard 24,Strawther 9,Nembhard 6,NaN
8,"Fri, Nov 26",vs 5 Duke *,L84-81,6-1 (0-0),Strawther 20,Strawther 10,Nembhard 11,NaN
9,"Mon, Nov 29",vs Tarleton,W64-55,7-1 (0-0),Bolton 17,Strawther 12,Holmgren 2,NaN


In [6]:
import requests
from bs4 import BeautifulSoup
import re
team_schedule_response = requests.get(load_database_functions.TEAM_SCHEDULE_URL.replace('TEAM_ID', str(2250)))
soup = BeautifulSoup(team_schedule_response.content, 'lxml')

table = soup.find('table')
table

<table class="Table" style="border-collapse:collapse;border-spacing:0"><colgroup class="Table__Colgroup"><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/></colgroup><thead class="Table__THEAD"></thead><tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="Table__Title pt5 Table__TD" colspan="8">Regular Season</td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="1"><td class="ttu small-col Table_Headers Table__TD"><span class="fw-medium w-100 dib ttu small-col Table_Headers">DATE</span></td><td class="ttu opponent-col Table_Headers Table__TD"><span class="fw-medium w-100 dib ttu opponent-col Table_Headers">OPPONENT</span></td><td class="ttu small-col Table_Headers Table__TD"><span class="fw-medium w-100 dib ttu small-col Table_Headers">RESULT</span></td><td class="ttu small-col Table_

In [7]:
# - Team_Schedule
#     - espn_team_id
#     - opponent_espn_team_id
#     - game_date (strftime("%m-%d-%y"))
#     - is_home
#     - is_neutral_court
#     - is_win
#     - final_score
#     - opponent_final_score
#     - game_time 
#     - tv
#     - cancelled_or_postponed

team_id = 2250
game_is_final = True
date_header_count = 0
all_games = []

for row in table.find_all('tr'):
    game_cancelled_or_postponed = False
    all_cols = row.find_all('td')
    if all_cols[0].text=='DATE':
        date_header_count+=1

    if date_header_count == 2:
        game_is_final = False

    if len(all_cols)>1 and all_cols[0].text!='DATE':
        print(all_cols[0].text)
        if all_cols[2].text.lower() == 'canceled' or all_cols[2].text.lower() == 'postponed':
            game_cancelled_or_postponed = True
        
        this_game = {'espn_team_id': team_id}
        col_count = 1
        
        for col in all_cols:
            if col_count == 1:
                #date
                game_date = datetime.datetime.strptime(col.text, "%a, %b %d")
                if game_date.date() < datetime.date(1900,5,1):
                    game_date = game_date + datetime.timedelta(365.25*122)
                else:
                    game_date = game_date + datetime.timedelta(365.25*121)


                this_game['game_date'] = game_date.strftime("%m-%d-%y")
            elif col_count == 2:
                #opponent_espn_team_id
                #is_home
                #is_neutral_court
                spans = col.find_all('span')
                this_game['is_home'] = spans[0].text == 'vs' and '*' not in spans[2].text
                this_game['opponent_espn_team_id'] = int(re.search(r'_/id/([0-9]+)', spans[1].find('a').attrs['href']).group(1))
                this_game['is_neutral_court'] = '*' in spans[2].text 
            elif col_count == 3:
                if game_is_final:
                    this_game['cancelled_or_postponed'] = game_cancelled_or_postponed
                    if game_cancelled_or_postponed:
                        this_game['is_win'] = None
                        this_game['final_score'] = None
                        this_game['opponent_final_score'] = None
                        this_game['game_time'] = None
                        this_game['tv'] = None
                        
                    else:
                        #is_win
                        #final_score
                        #opponent_final_score
                        spans = col.find_all('span')
                        this_game['is_win'] = spans[0].text == 'W'
                        if this_game['is_win']:
                            this_game['final_score'] = spans[1].text.split('-')[0]
                            this_game['opponent_final_score'] = spans[1].text.split('-')[1]
                        else:
                            this_game['final_score'] = spans[1].text.split('-')[1]
                            this_game['opponent_final_score'] = spans[1].text.split('-')[0]
                        
                        this_game['game_time'] = None
                        this_game['tv'] = None

                else:
                    # game_time 
                    # tv
                    spans = col.find_all('span')
                    this_game['game_time'] = spans[0].text
                    this_game['tv'] = None
                    
                    this_game['is_win'] = None
                    this_game['final_score'] = None
                    this_game['opponent_final_score'] = None

            col_count +=1
        all_games.append(this_game)

df = pd.DataFrame(all_games)
df

Tue, Nov 9
Sat, Nov 13
Mon, Nov 15
Fri, Nov 19
Mon, Nov 22
Tue, Nov 23
Fri, Nov 26
Mon, Nov 29
Sat, Dec 4
Thu, Dec 9
Sun, Dec 12
Sat, Dec 18
Mon, Dec 20
Tue, Dec 28
Thu, Dec 30
Sat, Jan 1
Sat, Jan 8
Thu, Jan 13
Sat, Jan 15
Thu, Jan 20
Thu, Jan 27
Sat, Jan 29
Thu, Feb 3
Sat, Feb 5
Thu, Feb 10
Sat, Feb 12
Wed, Feb 16
Sat, Feb 19
Thu, Feb 24
Sat, Feb 26


,espn_team_id,game_date,is_home,opponent_espn_team_id,is_neutral_court,cancelled_or_postponed,is_win,final_score,opponent_final_score,game_time,tv
0,2250,11-09-21,True,3101,False,False,True,97,63,None,None
1,2250,11-13-21,True,251,False,False,True,86,74,None,None
2,2250,11-15-21,True,2016,False,False,True,84,57,None,None
3,2250,11-19-21,True,91,False,False,True,92,50,None,None
4,2250,11-22-21,False,2117,True,False,True,107,54,None,None
5,2250,11-23-21,False,26,True,False,True,83,63,None,None
6,2250,11-26-21,False,150,True,False,False,81,84,None,None
7,2250,11-29-21,True,2627,False,False,True,64,55,None,None
8,2250,12-04-21,False,333,True,False,False,82,91,None,None
9,2250,12-09-21,True,2771,False,False,True,80,55,None,None


In [8]:
df.to_sql('team_schedule', con,if_exists='append', index=False)

30

In [12]:
sql_query = f'SELECT * FROM team_schedule WHERE espn_team_id = {team_id}'
df = pd.read_sql_query(sql_query, con)
df

,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
0,2250,3101,11-09-21,1,0,1.0,97.0,63.0,None,None,0.0
1,2250,251,11-13-21,1,0,1.0,86.0,74.0,None,None,0.0
2,2250,2016,11-15-21,1,0,1.0,84.0,57.0,None,None,0.0
3,2250,91,11-19-21,1,0,1.0,92.0,50.0,None,None,0.0
4,2250,2117,11-22-21,0,1,1.0,107.0,54.0,None,None,0.0
5,2250,26,11-23-21,0,1,1.0,83.0,63.0,None,None,0.0
6,2250,150,11-26-21,0,1,0.0,81.0,84.0,None,None,0.0
7,2250,2627,11-29-21,1,0,1.0,64.0,55.0,None,None,0.0
8,2250,333,12-04-21,0,1,0.0,82.0,91.0,None,None,0.0
9,2250,2771,12-09-21,1,0,1.0,80.0,55.0,None,None,0.0


In [13]:
df[df['cancelled_or_postponed'].isnull()]

,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
27,2250,2541,02-19-22,1,0,NaN,NaN,NaN,LIVE,None,NaN
28,2250,2539,02-24-22,0,0,NaN,NaN,NaN,9:00 PM,None,NaN
29,2250,2608,02-26-22,0,0,NaN,NaN,NaN,10:00 PM,None,NaN


In [49]:
if game_date.date() < datetime.date(1900,5,1):
    game_date = game_date + datetime.timedelta(365.25*122)
else:
    game_date = game_date + datetime.timedelta(365.25*121)

game_date

datetime.datetime(2022, 2, 26, 12, 0)

In [26]:
test_str = "/mens-college-basketball/team/_/id/2117/central-michigan-chippewas"


import re

find_id = re.compile(r'_/id/([0-9]+)')

find_id.findall(test_str)


'*' in 'Central Michigan *'

True

In [14]:
existing_games_remaining = df[df['cancelled_or_postponed'].isnull()].copy()
existing_games_remaining


,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
27,2250,2541,02-19-22,1,0,NaN,NaN,NaN,LIVE,None,NaN
28,2250,2539,02-24-22,0,0,NaN,NaN,NaN,9:00 PM,None,NaN
29,2250,2608,02-26-22,0,0,NaN,NaN,NaN,10:00 PM,None,NaN


In [15]:
latest_games_remaining = df[(df['cancelled_or_postponed'].isnull())& (df['game_time']!='LIVE')].copy()
latest_games_remaining

,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
28,2250,2539,02-24-22,0,0,NaN,NaN,NaN,9:00 PM,None,NaN
29,2250,2608,02-26-22,0,0,NaN,NaN,NaN,10:00 PM,None,NaN


In [18]:
lastest_df = df.copy()

lastest_df.loc[(lastest_df['game_time']=='LIVE'), ['is_win', 'final_score', 'opponent_final_score', 'cancelled_or_postponed']] = [1,99,65,0]
lastest_df

,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
0,2250,3101,11-09-21,1,0,1.0,97.0,63.0,None,None,0.0
1,2250,251,11-13-21,1,0,1.0,86.0,74.0,None,None,0.0
2,2250,2016,11-15-21,1,0,1.0,84.0,57.0,None,None,0.0
3,2250,91,11-19-21,1,0,1.0,92.0,50.0,None,None,0.0
4,2250,2117,11-22-21,0,1,1.0,107.0,54.0,None,None,0.0
5,2250,26,11-23-21,0,1,1.0,83.0,63.0,None,None,0.0
6,2250,150,11-26-21,0,1,0.0,81.0,84.0,None,None,0.0
7,2250,2627,11-29-21,1,0,1.0,64.0,55.0,None,None,0.0
8,2250,333,12-04-21,0,1,0.0,82.0,91.0,None,None,0.0
9,2250,2771,12-09-21,1,0,1.0,80.0,55.0,None,None,0.0


In [16]:
updated_dates = list(existing_games_remaining[~existing_games_remaining['game_date'].isin(latest_games_remaining['game_date'])]['game_date'])
updated_dates

['02-19-22']

In [31]:
#existing_games_remaining = existing_df[existing_df['cancelled_or_postponed'].isnull()].copy()
#latest_games_remaining = lastest_df[lastest_df['cancelled_or_postponed'].isnull()].copy()
lastest_df = df.copy()

lastest_df.loc[(lastest_df['game_time']=='LIVE'), ['is_win', 'final_score', 'opponent_final_score', 'cancelled_or_postponed']] = [1,99,65,0]
lastest_df

existing_games_remaining = df[df['cancelled_or_postponed'].isnull()].copy()
latest_games_remaining = df[(df['cancelled_or_postponed'].isnull()) & (df['game_time']!='LIVE')].copy()


if len(existing_games_remaining) > len(latest_games_remaining):
    #get dates that were updated and replace with new values
    updated_dates = list(existing_games_remaining[~existing_games_remaining['game_date'].isin(latest_games_remaining['game_date'])]['game_date'])

    for date in updated_dates:
        updated_date_value = lastest_df[lastest_df['game_date']== date]

        cur = con.cursor()

        update_str = f"""UPDATE team_schedule
                            SET is_win = {updated_date_value['is_win'].values[0]},
                                final_score = {updated_date_value['final_score'].values[0]},
                                opponent_final_score = {updated_date_value['opponent_final_score'].values[0]},
                                cancelled_or_postponed = {updated_date_value['cancelled_or_postponed'].values[0]}
                        WHERE
                            espn_team_id = {team_id} and
                            game_date = '{date}'"""
        print(update_str)

        cur.execute(update_str)
        con.commit()


UPDATE team_schedule
                            SET is_win = 1.0,
                                final_score = 99.0,
                                opponent_final_score = 65.0,
                                cancelled_or_postponed = 0.0
                        WHERE
                            espn_team_id = 2250 and
                            game_date = '02-19-22'


In [25]:
int(updated_date_value['is_win'].values[0])

1

In [30]:
sql_query = f'SELECT * FROM team_schedule WHERE espn_team_id = {team_id}'
df_new = pd.read_sql_query(sql_query, con)
df_new 


,espn_team_id,opponent_espn_team_id,game_date,is_home,is_neutral_court,is_win,final_score,opponent_final_score,game_time,tv,cancelled_or_postponed
0,2250,3101,11-09-21,1,0,1.0,97.0,63.0,None,None,0.0
1,2250,251,11-13-21,1,0,1.0,86.0,74.0,None,None,0.0
2,2250,2016,11-15-21,1,0,1.0,84.0,57.0,None,None,0.0
3,2250,91,11-19-21,1,0,1.0,92.0,50.0,None,None,0.0
4,2250,2117,11-22-21,0,1,1.0,107.0,54.0,None,None,0.0
5,2250,26,11-23-21,0,1,1.0,83.0,63.0,None,None,0.0
6,2250,150,11-26-21,0,1,0.0,81.0,84.0,None,None,0.0
7,2250,2627,11-29-21,1,0,1.0,64.0,55.0,None,None,0.0
8,2250,333,12-04-21,0,1,0.0,82.0,91.0,None,None,0.0
9,2250,2771,12-09-21,1,0,1.0,80.0,55.0,None,None,0.0


In [12]:
cur = con.cursor()
cur.execute('DELETE FROM team_week_rankings')
con.commit()

In [6]:
teams_df, team_week_rankings, existing_team_week_rankings = load_database_functions.load_top25_teams(con, 2)

In [7]:
team_week_rankings

,espn_team_id,week_number,ranking
0,2250,2,1
1,26,2,2
2,2305,2,3
3,130,2,4
4,222,2,5
5,2509,2,6
6,150,2,7
7,251,2,8
8,239,2,9
9,356,2,10


In [8]:
existing_team_week_rankings

,espn_team_id,week_number,ranking
0,2250,1,1
1,26,1,2
2,2305,1,3
3,222,1,4
4,251,1,5
5,130,1,6
6,2509,1,7
7,239,1,8
8,150,1,9
9,96,1,10


In [7]:
all_teams = pd.read_sql_query('''SELECT DISTINCT opponent_espn_team_id
FROM team_schedule as ts
LEFT JOIN teams as t on ts.opponent_espn_team_id=t.espn_team_id
WHERE t.espn_team_id is NULL''', con)

all_teams

,opponent_espn_team_id
0,3101
1,2016
2,91
3,2117
4,2627
...,...
245,2046
246,2174
247,2253
248,2239


In [22]:
for team_id in all_teams['opponent_espn_team_id']:
    print(team_id)
    team_response = requests.get(load_database_functions.TEAM_SCHEDULE_URL.replace('TEAM_ID', str(team_id)))

    soup = BeautifulSoup(team_response.content, 'lxml')
    team_name = soup.find('h1', attrs={'class': 'ClubhouseHeader__Name' }).find('span').find('span').text.replace("'", "''")
    logo_url = load_database_functions.LOGO_URL.replace('TEAM_ID', str(team_id))
    team_page_url = 'testURL' #load_database_functions.TEAM_HOME_URL.replace('TEAM_ID', str(team_id))

    query_str = f"INSERT INTO teams VALUES ({team_id},'{team_name}', '{logo_url}', '{team_page_url}')"
    cur.execute(query_str)
con.commit()

3101
2016
91
2117
2627
2771
264
2464
2453
301
2351
2492
2541
2539
2501
279
2934
299
2454
2439
38
2011
13
9
25
204
254
24
265
2619
249
2168
314
2599
2638
142
2617
2440
2244
197
277
201
2306
2628
116
47
2325
219
2603
183
156
218
305
2086
46
2277
2458
23
2856
2534
292
2029
2916
2084
2504
21
158
135
253
2870
2116
164
84
77
213
2294
282
2750
2437
152
2447
2110
2670
2466
349
2097
2241
322
2643
2569
2026
2210
259
228
87
59
2390
154
97
103
221
2523
195
399
2582
98
2272
61
238
245
344
2579
145
2296
2032
2132
2598
50
2635
2428
139
2655
2724
202
151
2567
58
2640
149
27
2348
2571
6
2473
2181
55
2275
242
2803
309
326
2382
2460
2031
2429
2006
315
189
119
2443
85
2630
2193
2506
2430
2908
2352
225
232
2427
231
104
288
2514
45
261
257
2329
2413
2433
43
2561
2099
68
2154
111
2230
2184
113
227
2115
112358
2066
2379
2755
2426
2515
256
161
270
2653
2463
526
2198
300
2449
16
325
3084
2623
328
2692
2335
2065
2681
2309
2450
2050
338
2545
2130
2711
2199
2649
2597
2739
2287
2377
2057
198
71
167
2005
278
2320
33

In [19]:
soup.find('h1', attrs={'class': 'ClubhouseHeader__Name' }).find('span').find('span').text

'Dixie State'

In [40]:
non_number_teams =pd.read_sql_query('''SELECT *
FROM team_schedule''', con)
#non_number_teams['opponent_espn_team_id'].astype(int)

mask = pd.to_numeric(non_number_teams['opponent_espn_team_id'], errors='coerce').isna() 

non_number_teams.loc[mask, 'opponent_espn_team_id']

def get_id(opponent_espn_team_id):
    print(opponent_espn_team_id)
    print(-int(''.join([str(ord(chr_str) - 96) for chr_str in opponent_espn_team_id])))
    return -int(''.join([str(ord(chr_str) - 96) for chr_str in opponent_espn_team_id]))


non_number_teams.loc[mask, 'new_id'] = non_number_teams.loc[mask].apply(lambda x: get_id(x['opponent_espn_team_id']), axis=1)
non_number_teams


Chaminade
298113914145
Central Methodist


ValueError: invalid literal for int() with base 10: '-295142018112-64-19520815491920'

In [43]:
smallest_id = pd.read_sql_query('''SELECT min(espn_team_id)
FROM teams''', con)
smallest_id.values[0][0]

2

In [3]:
smallest_id = cur.execute('SELECT min(espn_team_id) FROM teams')
smallest_id.fetchone()[0]

-13

In [54]:
#cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
cur.execute("INSERT INTO teams VALUES (-1,'Test2', NULL, NULL)")

In [5]:
all_teams = pd.read_sql_query("""SELECT DISTINCT t.espn_team_id 
                                 FROM teams as t 
                                 INNER JOIN team_week_rankings as twr 
                                 on t.espn_team_id= twr.espn_team_id""", con)
all_teams

,espn_team_id
0,2250
1,26
2,2305
3,222
4,251
5,130
6,2509
7,239
8,150
9,96
